### Merge financial and sentiment dataset

In [1]:
import pandas as pd

In [4]:
dataset = pd.read_csv('sentiment_data/test_sentiment.csv').drop(columns='Unnamed: 0', inplace=False)
dataset.head(2)

,ticker_symbol,tweet_id,writer,text,comment_num,retweet_num,like_num,company_name,sector,year,month,day,hour,sentiment
0,V,1300774726432731137,itz¹¹⁷,RT @KTH_Facts: Taehyung a Baritone belting hig...,0,0,0,Visa Inc.,Information Technology,2020,9,1,12,1
1,V,1306878046927028224,📖TheLindoPress¹¹⁷,RT @pinkvilla: #BTSARMY recently trended KTH1 ...,0,0,0,Visa Inc.,Information Technology,2020,9,18,8,1


In [5]:
symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']

for symbol in symbols:

    price = pd.read_csv(f'financial_data/{symbol}.csv')
#     sent = pd.read_csv(f'sentiment_data/{symbol}_sentiment.csv')
    sent = dataset[dataset.ticker_symbol==symbol].copy()

    # create DATE column into an object
#     sent['Date'] = sent.apply(lambda x: str(x[10])+"-"+str(x[11])+"-"+str(x[12]), axis=1)
    sent['Date'] = sent.apply(lambda x: str(x[9])+"-"+str(x[10])+"-"+str(x[11]), axis=1)
    sent['Date'] = pd.to_datetime(sent["Date"], format='%Y-%m-%d', infer_datetime_format=True)
    sent['Date'] = sent.Date.apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # negative sentiment
    df0 = sent[sent['sentiment']==0].groupby('Date').count().reset_index()
    # positive sentiment
    df1 = sent[sent['sentiment']==1].groupby('Date').count().reset_index()

    # merge sentiments
    df = pd.merge(df1[['Date', 'sentiment']], df0[['Date', 'sentiment']], on='Date', how='outer', 
                  suffixes=('_pos', '_neg'))
    
    # merge with financial data
    df = pd.merge(price, df, on='Date', how='left')
    df = df.sort_values('Date')
    
    # keep data earlier than 2020
    df = df[df.Date.str[:4].astype(int)<2020]
    
    display(df.head(3))
    # save to csv into the root
    df.to_csv(f'{symbol}.csv', index=False)
    print(f'{symbol}.csv was created')

,Date,Open,High,Low,Close,Adj Close,Volume,sentiment_pos,sentiment_neg
0,2015-01-02,27.8475,27.860001,26.837500,27.332500,24.861401,212818400,618.0,226.0
1,2015-01-05,27.0725,27.162500,26.352501,26.562500,24.161013,257142000,868.0,257.0
2,2015-01-06,26.6350,26.857500,26.157499,26.565001,24.163294,263188400,865.0,265.0


AAPL.csv was created


,Date,Open,High,Low,Close,Adj Close,Volume,sentiment_pos,sentiment_neg
0,2015-01-02,532.599976,535.799988,527.880005,529.549988,529.549988,1324000,95.0,18.0
1,2015-01-05,527.150024,527.989990,517.750000,519.460022,519.460022,2059100,208.0,18.0
2,2015-01-06,520.500000,521.210022,505.549988,506.640015,506.640015,2722800,256.0,25.0


GOOGL.csv was created


,Date,Open,High,Low,Close,Adj Close,Volume,sentiment_pos,sentiment_neg
0,2015-01-02,46.660000,47.419998,46.540001,46.759998,41.443684,27913900,77.0,39.0
1,2015-01-05,46.369999,46.730000,46.250000,46.330002,41.062580,39673900,77.0,28.0
2,2015-01-06,46.380001,46.750000,45.540001,45.650002,40.459892,36447900,56.0,16.0


MSFT.csv was created


,Date,Open,High,Low,Close,Adj Close,Volume,sentiment_pos,sentiment_neg
0,2015-01-02,44.574001,44.650002,42.652000,43.862000,43.862000,23822000,144.0,52.0
1,2015-01-05,42.910000,43.299999,41.431999,42.018002,42.018002,26842500,209.0,71.0
2,2015-01-06,42.012001,42.840000,40.841999,42.256001,42.256001,31309500,171.0,54.0


TSLA.csv was created


,Date,Open,High,Low,Close,Adj Close,Volume,sentiment_pos,sentiment_neg
0,2015-01-02,312.579987,314.750000,306.959991,308.519989,308.519989,2783200,117.0,51.0
1,2015-01-05,307.010010,308.380005,300.850006,302.190002,302.190002,2774200,214.0,51.0
2,2015-01-06,302.239990,303.000000,292.380005,295.290009,295.290009,3519000,153.0,41.0


AMZN.csv was created
